# 21M.387 Fundamentals of Music Processing
## Lab4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
from ipywidgets import interact
import sys
sys.path.append("..")
import fmplib as fmp
from fmplib.pyqueue import connect_to_queue

plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['image.interpolation'] = 'nearest'
fmp.documentation_button()

## Python Review

Useful python tricks below!


### enumerate
Use [`enumerate`](https://docs.python.org/3/library/functions.html#enumerate) to automatically generate an index inside a `for` loop:

In [ ]:
numbers = 2 ** np.arange(8)
for (idx, n) in enumerate(numbers):
    print(f'{idx}: {n}')


### list comprehension

Use [List comprehension](http://www.secnetix.de/olli/Python/list_comprehensions.hawk) to create arrays without a `for` loop using a more compact syntax.

In [ ]:
word_list = ['a', 'list', 'of', 'some', 'words']

# create a new list where each item is modified:
new_array = [x + '!' for x in word_list]
print(new_array)

# can be used to filter as well:
new_array = [w for w in word_list if len(w) == 4]
print(new_array)

### np.dot

Use `np.dot` for matrix multiplication and `.T` to transpose a matrix. For example:

In [ ]:
a = np.array(((1,2), (3,4), (5,6)))
b = np.array(((1,-1), (1,-1), (1,-1)))
print('A')
print(a, '\n')
print('B')
print(b, '\n')

print('A^T . B')
print(np.dot(a.T, b), '\n')
print('A . B^T')
print(np.dot(a, b.T))

### np.tile

Use [`np.tile`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.tile.html) to created an extended, tiled matrix (either vertically or horizontally or both)


In [ ]:
a = np.arange(4).reshape(2,2)
b = np.arange(18).reshape(6,3)
print('A')
print(a, '\n')

a_tiled1 = np.tile(a, (1, 4))
print('A_t1')
print(a_tiled1, '\n')

a_tiled2 = np.tile(a, (3, 1))
print('A_t2')
print(a_tiled2)

### np.argmax and np.argmin

Use [`np.argmax`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) to find the _index_ of the maximum value of an array.  
`np.argmin` is the same idea but uses the minimum value.
    

In [ ]:
a = 10 * np.random.random( 6 )
print(f'a: {a} \n')

idx = np.argmax(a)
print(f'argmax(a) = {idx}')
print(f'max(a) = {a[idx]}')

## Exercise 1

For matrix $\mathbf{A}$ below, find:
- the maximum value along each column, as a vector
- the minimum value along each row, as a vector

(hint: look up `np.min` and `np.max`).

After you have the two vectors:
- Divide $\mathbf{A}$ by the max row vector (column-wise divide)
- Subtract from $\mathbf{A}$ the min column vector (row-wise subtraction)

In [ ]:
a = np.roll(np.arange(15).reshape(3,5), 2)
print('A')
print(a)

In [ ]:
# a_max = 
# a_min = 

In [ ]:
connect_to_queue()

## Exercise 2

Write `normalize_matrix()` to normalize the _columns_ of a matrix using the $L^2$ norm. Hint: look at [`np.linalg.norm`](https://numpy.org/doc/stable/reference/routines.linalg.html).

Test your function on matrix $\mathbf{A}$ above by showing that the normalized matrix columns are all length 1.


In [ ]:
def normalize_matrix(mtx):
    pass

# TODO normalize `a`

# TODO verify that all columns of `a` are length 1


In [ ]:
connect_to_queue()

In [ ]:
connect_to_queue()

## Exercise 3

The next two exercises represent some of the initial steps needed to create a chromagram. We look at finding the FFT bins associated with a set of pitches. 

Write the function `bins_of_pitch()` which takes a single midi-pitch (scalar only) and returns an array of the frequency bins (i.e, values of $k$) that contribute to $\pm 0.5$ semitones around that pitch.

Inputs are: 
- `p`: MIDI pitch
- `fs`: $F_s$, the sampling frequency of the signal
- `fft_len`: $N$, the length of the DFT

Test your function below.

In [ ]:
def bins_of_pitch(p, fs, fft_len):
    pass

In [ ]:
print(bins_of_pitch(69, 22050, 2048))
# should return: [40, 41, 42]

print(bins_of_pitch(60, 22050, 4096))
# should return: [48, 49, 50]


In [ ]:
connect_to_queue()

## Exercise 4

Because the frequency resolution of a DFT can be poor for low notes, it is important to pick an analysis window size that provides enough frequency resolution to create a good chromagram.

For example, see what happens when you look for the frequency bins associated with the note _E2_ (pitch = 40) when $F_s = 22050$, and the DFT size is $N = 2048$

In [ ]:
# bins_of_pitch(?, ?, ?)


You should see an empty array (no frequency bins for that pitch!).

What value of $N$ (with $N$ being a power of 2) is needed to ensure that all pitches from _C2_ (pitch = 36) on up will have at least two contributing frequency bins?

You can approach this problem experimentally, by trying increasing values of $N$ and testing against a range of pitch values.

In [ ]:
connect_to_queue()

## Exercise 5

The following are two recordings of the beginning of Bach's First Prelude, played at identical tempos. However, they are played in different keys!

Listen to the audio. Can you hear which key each is played in? If not, don't worry. We will use TECHNOLOGY to help us figure it out.

In [ ]:
fs = 22050
bach1 = fmp.load_wav('audio/prelude_01.wav')
ipd.Audio(bach1, rate=fs)

In [ ]:
bach2 = fmp.load_wav('audio/prelude_02.wav')
ipd.Audio(bach2, rate=fs)

Create chromagrams of each signal with `fmp.make_chromagram()`. See fmplib documentation.
You can use these params:
- window length: $N = 4096$
- hop size: $H = 2048$

Then view and compare these chromagrams (`plt.imshow()` with `origin='lower'` and `aspect='auto'` will work).
Can you tell by visual inspection what key each piece is played in?

## Exercise 6

We can also answer the question by computational analysis to find the "semitone difference" between these two recordings. If we know that recording 1 is played in C (which it is), we can then calculate the most likely key of recording 2.

The strategy is to compare chromagram 1 with 12 "transposed versions" of chromagram 2. The one that has the highest similarity score represents the most likely semitone difference.

How do we compare chromagrams? In this case, the problem is made easier because both chromagrams have the same number of columns (ie, same length in time). Recall that dot products are useful ways of comparing two vectors to arrive at a similarity score.

[`np.roll`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.roll.html) is a handy function for this purpose.

Write the function `pitch_difference(chroma1, chroma2)` that returns the most likely semitone pitch transposition of chroma2 compared to chroma1.

In [ ]:
def pitch_difference(chroma1, chroma2):
    pass

In [ ]:
connect_to_queue('checkoff')